# <font color=blue>Assignments for "Overfitting and Regularization"</font>

In this assignment, you'll continue working with the house prices data. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

In [ ]:
# These are the libraries we use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.model_selection import GridSearchCV




import warnings
warnings.filterwarnings('ignore')
house_prices_df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
house_prices_df.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


- Reimplement your model from the previous lesson.

In [ ]:
df_null = house_prices_df.isna().sum() / len(house_prices_df)
house_prices_df.drop(df_null[df_null > .1].index, axis=1, inplace=True)
house_prices_df.dropna(inplace=True)

cols=['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt','YearRemodAdd']

df_train = house_prices_df[cols]

MSZoning = pd.get_dummies(house_prices_df['MSZoning'], drop_first=True)
Street = pd.get_dummies(house_prices_df['Street'], drop_first=True)
BsmtQual = pd.get_dummies(house_prices_df['BsmtQual'], drop_first=True)
SaleCondition = pd.get_dummies(house_prices_df['SaleCondition'], drop_first=True)

scaler = StandardScaler()

df_train = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns, index=df_train.index)

X = pd.concat([df_train, MSZoning, Street, BsmtQual,SaleCondition],axis =1)

y = house_prices_df['SalePrice']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
lrm = LinearRegression()
lrm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

In [ ]:
print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8269380440883238
-----Test set statistics-----
R-squared of the model in test set is: 0.515267553440401
Mean absolute error of the prediction is: 23365.348118051574
Mean squared error of the prediction is: 2269430398.335494
Root mean squared error of the prediction is: 47638.53900294901
Mean absolute percentage error of the prediction is: 14.199151194424855


- Try OLS, Lasso, Ridge and ElasticNet regressions using the same model specification. This time, you need to do **k-fold cross-validation** to choose the best hyperparameter values for your models. Which model is the best? Why?

### OLS

In [ ]:
import statsmodels.api as sm

results = sm.OLS(y_train, X_train).fit()

display(results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.971
Model:                            OLS   Adj. R-squared (uncentered):              0.970
Method:                 Least Squares   F-statistic:                              1744.
Date:                Mon, 31 May 2021   Prob (F-statistic):                        0.00
Time:                        14:05:40   Log-Likelihood:                         -12708.
No. Observations:                1070   AIC:                                  2.546e+04
Df Residuals:                    1050   BIC:                                  2.556e+04
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
OverallQual   2.207e+04   1793.310     12.306      0.000    1.85e+04    2.56e+04
GrLivArea     3.331e+04   1749.698     19.038      0.000    2.99e+04    3.67e+04
GarageCars    7432.4300   1554.985      4.780      0.000    4381.197    1.05e+04
TotalBsmtSF   1.615e+04   1483.568     10.887      0.000    1.32e+04    1.91e+04
FullBath     -8145.1697   1654.987     -4.922      0.000   -1.14e+04   -4897.711
YearBuilt     4276.5686   2038.381      2.098      0.036     276.805    8276.332
YearRemodAdd  6123.8539   1466.376      4.176      0.000    3246.492    9001.215
FV            8.695e+04   1.71e+04      5.082      0.000    5.34e+04    1.21e+05
RH             8.16e+04   1.92e+04      4.239      0.000    4.38e+04    1.19e+05
RL            8.777e+04   1.63e+04      5.394      0.000    5.58e+04     1.2e+05
RM             7.26e+04   1.63e+04      4.449      0.000    4.06e+04    1.05e+05
Pave          1.215e+05   1.61e+04      7.566      0.000       9e+04    1.53e+05
Fa           -3.706e+04   9137.223     -4.056      0.000    -5.5e+04   -1.91e+04
Gd           -3.754e+04   4482.976     -8.373      0.000   -4.63e+04   -2.87e+04
TA            -3.78e+04   5664.511     -6.673      0.000   -4.89e+04   -2.67e+04
AdjLand       3.148e+04   3.55e+04      0.887      0.375   -3.82e+04    1.01e+05
Alloca        4.215e+04   1.64e+04      2.576      0.010       1e+04    7.43e+04
Family         739.7969   9914.998      0.075      0.941   -1.87e+04    2.02e+04
Normal        1.517e+04   4592.992      3.304      0.001    6162.100    2.42e+04
Partial       3.048e+04   6071.676      5.021      0.000    1.86e+04    4.24e+04
==============================================================================
Omnibus:                      298.811   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            25769.646
Skew:                          -0.001   Prob(JB):                         0.00
Kurtosis:                      27.042   Cond. No.                         68.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Ridge

In [ ]:
from sklearn.linear_model import Ridge
ridgeregr = Ridge(alpha=1000) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.7417394407165754
-----Test set statistics-----
R-squared of the model in test set is: 0.6610163879144781
Mean absolute error of the prediction is: 24578.40981828036
Mean squared error of the prediction is: 1587060489.2752175
Root mean squared error of the prediction is: 39837.92777335711
Mean absolute percentage error of the prediction is: 14.260817214191329


In [ ]:
params = {'alpha': [10**i for i in range(1, 10)] + [round(0.1**i, 10) for i in range(10)]}


In [ ]:
ridge_grid = GridSearchCV(Ridge(), param_grid=params, cv=10)

ridge_grid.fit(X_train, y_train)
print(ridge_grid.best_params_)

pd.DataFrame(ridge_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

{'alpha': 1.0}


,param_alpha,mean_test_score
9,1,0.816488
0,10,0.815879
10,0.1,0.815364
11,0.01,0.814825
12,0.001,0.814745


### Lasso

In [ ]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=1000) 
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8119485062221735
-----Test set statistics-----
R-squared of the model in test set is: 0.5252171067247484
Mean absolute error of the prediction is: 24008.340403823426
Mean squared error of the prediction is: 2222848373.893727
Root mean squared error of the prediction is: 47147.09295273386
Mean absolute percentage error of the prediction is: 14.748283241568691


In [ ]:

lasso_grid = GridSearchCV(Lasso(), param_grid=params, cv=10)

lasso_grid.fit(X_train, y_train)
print(lasso_grid.best_params_)
pd.DataFrame(lasso_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

{'alpha': 100}


,param_alpha,mean_test_score
1,100,0.817061
0,10,0.816307
9,1,0.815003
10,0.1,0.814770
11,0.01,0.814745


### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet() 
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.7770854267597116
-----Test set statistics-----
R-squared of the model in test set is: 0.6365374264345531
Mean absolute error of the prediction is: 23710.303904354307
Mean squared error of the prediction is: 1701666597.6480243
Root mean squared error of the prediction is: 41251.26177037527
Mean absolute percentage error of the prediction is: 14.030102595445495


In [ ]:
elastic_grid = GridSearchCV(ElasticNet(), param_grid=params, cv=10)

elastic_grid.fit(X_train, y_train)
print(elastic_grid.best_params_)
pd.DataFrame(elastic_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

{'alpha': 0.01}


,param_alpha,mean_test_score
11,0.01,0.816636
12,0.001,0.816141
13,0.0001,0.815101
14,1e-05,0.814786
15,1e-06,0.814747
